# 3. Deep Learning

### Definición RNN
Usar capa SimpleRNN y la columna demanda como target.

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import pickle as pkl
import json

sys.path.append(os.path.abspath("../"))

import plotly.express as px

from encoding import encoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error, r2_score
from holidays import Spain
from StreamlitApp.functions.carga_dataframes import *
from StreamlitApp.passwords import pw
from ML.escalado_datos import *

from keras.layers import Input, SimpleRNN, Dense, Dropout, LSTM, GlobalMaxPool1D
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from rnn_lstm import *


In [2]:
metricas_rnn_lstm = {
    "RNN_1step": {
        "MSE": None,
        "MAE": None,
        "RMSE": None,
        "R2": None
    },
    "RNN_multistep": {
        "MSE": None,
        "MAE": None,
        "RMSE": None,
        "R2": None
    },
    "LSTM_1step": {
        "MSE": None,
        "MAE": None,
        "RMSE": None,
        "R2": None
    },
    "LSTM_multistep": {
        "MSE": None,
        "MAE": None,
        "RMSE": None,
        "R2": None
    }
}

## Preparación de los datos
Realizamos filtrado y escalado.

Vamos a usar un encoding circular, para que el modelo entienda mejor la estacionalidad de los datos.

Primero, definimos los días que tiene cada mes. Debemos hacerlo así, con un diccionario, ya que si intentamos hacer un groupby para sacar cuántos días tiene cada mes en el histórico (viendo los bisiestos), del mes actual solo cogerá el número de días que hayan pasado (por ejemplo, si tenemos datos hasta el 6 de marzo, escalará los datos de ese mes dividiendo entre 6...)

In [2]:
with open("../data/data_scaled/scalers/scaler_consumo_anio_DF_DEMANDA.pkl", "br") as file:
    scaler = pkl.load(file)

In [3]:
df_demanda = carga_dataframes(pw["host"], pw["user"], pw["password"], pw["database"])[1]
df = procesar_datos(df_demanda)

Columnas antes de procesar: Index(['fecha', 'año', 'mes', 'dia', 'dia_semana', 'valor_(GWh)',
       'es_festivo'],
      dtype='object')
Procesamiento completado. Datos preparados y guardados.


In [4]:
df = df.drop(columns=['fecha'])

Si lo representamos, sale un círculo de radio 1.

In [6]:
fig = px.line(
    data_frame=df[:32],
    x='dia_sin',
    y='dia_cos'
)

fig.show()

### Secuencias de entrada y salida
Las redes recurrentes aprenden observando una secuencia con sus características y prediciendo el siguiente valor del target (en este caso, la demanda). Lo que haremos será crear ventanas deslizantes de "loockback" días:
- En la X nos guardamos los datos de los 'loockback' días anteriores.
- En la y intentará predecir el día siguiente.
- Devuelve un array para cada una que podrá entrar a la red neuronal.

In [ ]:
# def create_sequences(df, target_column, lookback):
#     X, y = [], []
#     for i in range(len(df) - lookback):
#         X.append(df.iloc[i:i+lookback].drop(columns=[target_column]).to_numpy()) 
#         y.append(df.iloc[i+lookback][target_column]) 
#     return np.array(X), np.array(y)

# Definir los días que se van a usar para predecir

X, y = create_sequences_rnn(df, target_column="valor_(GWh)", lookback=60)

print("Forma de X:", X.shape)  # (n_samples, lookback, n_features)
print("Forma de y:", y.shape)  # (n_samples,)

Forma de X: (5115, 60, 8)
Forma de y: (5115,)


### Train/Test
Debe mantener la temporalidad: el 80% de los datos más antiguos irán al train set y el 20% restante al test

In [10]:
def train_test(f=0.8):    
    train_size = int(len(X) * f)

    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = train_test()

## Modelo RNN
Generamos la estructura de la red neuronal. 
+ En primer lugar usaremos una RNN simple y veremos la función de pérdida.
+ Luego aplicaremos una

### Definición RNN simple

In [13]:
rnn, history_rnn = get_model_rnn(X, X_train, X_val, y_train, y_val)

Epoch 1/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0315 - mae: 0.1383 - val_loss: 0.0100 - val_mae: 0.0761
Epoch 2/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0138 - mae: 0.0904 - val_loss: 0.0087 - val_mae: 0.0696
Epoch 3/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0117 - mae: 0.0832 - val_loss: 0.0088 - val_mae: 0.0713
Epoch 4/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0106 - mae: 0.0787 - val_loss: 0.0088 - val_mae: 0.0717
Epoch 5/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0099 - mae: 0.0750 - val_loss: 0.0082 - val_mae: 0.0690
Epoch 6/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0097 - mae: 0.0742 - val_loss: 0.0077 - val_mae: 0.0674
Epoch 7/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0084 - mae: 0.0690 - val_loss: 0.0079 - val_mae: 0.0687
Epoch 8/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0082 - mae: 0.0683 - val_loss: 0.0087 - val_mae: 0.0719
Epoch 9/100
128/128 ━━━━━━━━━━━━━━━━━━━━

In [14]:
fig_mae_rnn = plot_mae(history_rnn)

fig_mae_rnn.show()

### One-step predictions
Aquí nos cogeremos los datos de los últimos 14 días y con ellos haremos la predicción de los siguientes 14 de uno en uno.

In [ ]:
# model_rnn = joblib.load("MODELS/RNN_LSTM/rnn.pkl")

In [15]:
pred_1s, target_1s = predict_1step(X, y_val, rnn, scaler)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━

In [16]:
fig_val_rnn_1s = plot_validation(target_1s, pred_1s)
fig_val_rnn_1s.show()

In [18]:
metricas_rnn_1s = metricas(target_1s, pred_1s)

for elem, metrica in zip(metricas_rnn_1s, metricas_rnn_lstm["RNN_1step"]):
    metrica = elem

In [20]:
metricas_rnn_1s

(1891.98, 35.77, np.float64(43.5), 0.33)

In [19]:
metricas_rnn_lstm

{'RNN_1step': {'MSE': None, 'MAE': None, 'RMSE': None, 'R2': None},
 'RNN_multistep': {'MSE': None, 'MAE': None, 'RMSE': None, 'R2': None},
 'LSTM_1step': {'MSE': None, 'MAE': None, 'RMSE': None, 'R2': None},
 'LSTM_multistep': {'MSE': None, 'MAE': None, 'RMSE': None, 'R2': None}}

In [108]:
fig_one = px.line(
                       y=[validation_real_one, predictions_real_one],
                       title='Validación de las predicciones frente a valores reales',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_one.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_one.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_one.show()

In [58]:
mse = round(mean_squared_error(validation_real_one, predictions_real_one),2)
print(f"MSE: {mse}")

mae = round(mean_absolute_error(validation_real_one, predictions_real_one), 2)
print(f"MAE: {mae}")

rmse = round(np.sqrt(mse), 2)
print(f"RMSE: {rmse}")

r2 = round(r2_score(validation_real_one, predictions_real_one), 2)
print(f"R²: {r2}")

MSE: 2090.68
MAE: 37.43
RMSE: 45.72
R²: 0.55


In [98]:
# n_future = 14  # Número de días a predecir
# future_predictions = []

# # Última ventana de datos (últimos 'lookback' valores de X)
# last_x = X[-1].copy()  # Copia la última secuencia para no modificar X

# for _ in range(n_future):
#     # Predecir el siguiente valor
#     p = model_rnn.predict(last_x.reshape(1, X.shape[1], X.shape[2]))[0, 0]
    
#     # Guardar la predicción
#     future_predictions.append(p)
    
#     # Desplazar la ventana de entrada hacia atrás y agregar la predicción
#     last_x[:-1] = last_x[1:]  # Mueve los datos hacia atrás
#     last_x[-1, 0] = p  # Inserta la nueva predicción como último valor

# print(f"Predicciones para los próximos {n_future} días: {future_predictions}")

n_future = 14  # Número de días a predecir

# Usamos la misma última ventana en cada predicción
future_predictions = model_rnn.predict(X[-n_future:])

print(f"Predicciones para los próximos {n_future} días: {future_predictions.flatten()}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
Predicciones para los próximos 14 días: [0.5387787  0.5627171  0.54799056 0.511876   0.47505766 0.35414305
 0.28459954 0.5001829  0.5411366  0.51722336 0.5203417  0.5072141
 0.41794485 0.33258283]


In [99]:
one_real = y_val[-14:]#.tolist()

one_real, predictions_real_one = desescalado(one_real, future_predictions, scaler)

# Graficar
fig_one = px.line(
                       y=predictions_real_one,
                       title='Predicción de la demanda en 14 días',
                       labels={'x': 'Día', 'y': 'Demanda (GWh)'}

                       )

fig_one.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_one.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_one.show()


### Multi-step prediction
Aquí, también haremos la predicción de los siguientes 14 días, pero aprovechando en cada iteración los nuevos datos aportados por la predicción anterior.

In [76]:
validation_target_multi = y_val
validation_predictions_multi = []

# Usa la primera ventana de validación (X_val[0])
last_x = X_val[0].copy()  # La primera secuencia de validación

for i in range(len(validation_target_multi)):
    # Predicción del siguiente paso
    p = model_rnn.predict(last_x.reshape(1, X.shape[1], X.shape[2]))[0, 0]
    
    validation_predictions_multi.append(p)
    print(f"Paso {i+1} -> Valor real: {validation_target_multi[i]:.4f} | Predicción: {p:.4f}")

    # Desplaza la ventana hacia atrás e inserta la nueva predicción
    last_x[:-1] = last_x[1:]  # Mueve los datos hacia atrás
    last_x[-1, 0] = p  # Inserta la nueva predicción como último valor

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Paso 1 -> Valor real: 0.4389 | Predicción: 0.4351
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Paso 2 -> Valor real: 0.4110 | Predicción: 0.4731
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Paso 3 -> Valor real: 0.2460 | Predicción: 0.4866
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Paso 4 -> Valor real: 0.1406 | Predicción: 0.4617
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Paso 5 -> Valor real: 0.3698 | Predicción: 0.4361
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Paso 6 -> Valor real: 0.4229 | Predicción: 0.3412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Paso 7 -> Valor real: 0.4338 | Predicción: 0.3041
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Paso 8 -> Valor real: 0.4463 | Predicción: 0.3142
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Paso 9 -> Valor real: 0.4346 | Predicción: 0.3088
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Paso 10 -> Valor real: 0.2848 | Predicción: 0.3132
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Paso 11 -> Valor real: 0.1746 | Predicción: 0.3255
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74

In [79]:
validation_real_multi, predictions_real_multi = desescalado(validation_target_multi, validation_predictions_multi, scaler)

In [85]:
fig_multi = px.line(
                       y=[validation_real_multi, predictions_real_multi],
                       title='Validación de las predicciones frente a valores reales',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_multi.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_multi.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_multi.show()

In [75]:
mse = round(mean_squared_error(validation_real_multi, predictions_real_multi),2)
print(f"MSE: {mse}")

mae = round(mean_absolute_error(validation_real_multi, predictions_real_multi), 2)
print(f"MAE: {mae}")

rmse = round(np.sqrt(mse), 2)
print(f"RMSE: {rmse}")

r2 = round(r2_score(validation_real_multi, predictions_real_multi), 2)
print(f"R²: {r2}")

MSE: 11977.96
MAE: 90.06
RMSE: 109.44
R²: -1.57


In [ ]:
n_future = 14  # Número de días a predecir
future_predictions_multi = []

# Última ventana de datos (últimos 'lookback' valores de X)
last_x = X[-1].copy()  # Copia la última secuencia para no modificar X

for i in range(n_future):
    # Predicción del siguiente valor
    p = model_rnn.predict(last_x.reshape(1, X.shape[1], X.shape[2]))[0, 0]
    
    future_predictions_multi.append(p)
    print(f"Día {i+1} -> Predicción: {p:.4f}")

    # Desplazar la ventana hacia atrás e insertar la nueva predicción
    last_x[:-1] = last_x[1:]  # Mueve los datos hacia atrás
    last_x[-1, 0] = p  # Inserta la nueva predicción como último valor

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Día 1 -> Predicción: 0.3326
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Día 2 -> Predicción: 0.4754
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Día 3 -> Predicción: 0.4519
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Día 4 -> Predicción: 0.4840
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Día 5 -> Predicción: 0.4700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Día 6 -> Predicción: 0.4676
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Día 7 -> Predicción: 0.3985
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Día 8 -> Predicción: 0.3660
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Día 9 -> Predicción: 0.3453
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Día 10 -> Predicción: 0.3447
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Día 11 -> Predicción: 0.3209
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Día 12 -> Predicción: 0.3052
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Día 13 -> Predicción: 0.3326
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Día 14 -> Predicción: 0.3616


In [97]:
multi_real = y_val[-14:]#.tolist()

multi_real, predictions_real_multi = desescalado(multi_real, future_predictions_multi, scaler)

# Graficar
fig_one = px.line(
                       y=predictions_real_multi,
                       title='Predicción de la demanda en 14 días',
                       labels={'x': 'Día', 'y': 'Demanda (GWh)'}

                       )

fig_one.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_one.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_one.show()

### Función con ruido
Vamos a añadir algo de ruido a las predicciones, de forma que no sean tan rígidas y puedan ajustarse mejor a variabilidades no explicadas.

In [89]:
import numpy as np

validation_real_noise = y_val  # Última parte de los datos reales
validation_predictions_noise = []

# Usar la primera ventana de validación
last_x = X_val[0].copy()

sigma = 0.05  # Desviación estándar del ruido

for i in range(len(validation_real_noise)):  
    # Predicción sin ruido
    p = model_rnn.predict(last_x.reshape(1, last_x.shape[0], X_val.shape[2]))[0, 0]
    
    # Agregar ruido gaussiano proporcional
    noise = np.random.normal(loc=0, scale=sigma * abs(p), size=1)[0]  # Proporcional al valor predicho
    p_noisy = p + noise
    
    validation_predictions_noise.append(p_noisy)
    print(f"Paso {i+1} -> Valor real: {validation_real_noise[i]:.4f} | Predicción con ruido: {p_noisy:.4f}")

    # Desplazar la ventana y actualizar el último valor con la predicción ruidosa
    last_x[:-1] = last_x[1:]  # Mueve los datos hacia atrás
    last_x[-1, 0] = p_noisy  # Inserta la predicción con ruido como último valor


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Paso 1 -> Valor real: 0.4389 | Predicción con ruido: 0.4064
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Paso 2 -> Valor real: 0.4110 | Predicción con ruido: 0.4851
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Paso 3 -> Valor real: 0.2460 | Predicción con ruido: 0.4906
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Paso 4 -> Valor real: 0.1406 | Predicción con ruido: 0.4656
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Paso 5 -> Valor real: 0.3698 | Predicción con ruido: 0.4495
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Paso 6 -> Valor real: 0.4229 | Predicción con ruido: 0.3484
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Paso 7 -> Valor real: 0.4338 | Predicción con ruido: 0.3154
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Paso 8 -> Valor real: 0.4463 | Predicción con ruido: 0.3231
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Paso 9 -> Valor real: 0.4346 | Predicción con ruido: 0.3191
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Paso 10 -> Valor real: 0.2848 | Predicción con ruido: 0.3051
1/1 ━━━━━━━━━━━━━

In [90]:
validation_real_noise, predictions_real_noise = desescalado(validation_real_noise, validation_predictions_noise, scaler)

In [91]:
fig_noise = px.line(
                       y=[validation_real_noise, predictions_real_noise],
                       title='Predicción de la demanda en 14 días',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_noise.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_noise.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_noise.show()

In [92]:
mse = round(mean_squared_error(validation_real_noise, predictions_real_noise),2)
print(f"MSE: {mse}")

mae = round(mean_absolute_error(validation_real_noise, predictions_real_noise), 2)
print(f"MAE: {mae}")

rmse = round(np.sqrt(mse), 2)
print(f"RMSE: {rmse}")

r2 = round(r2_score(validation_real_noise, predictions_real_noise), 2)
print(f"R²: {r2}")

MSE: 7338.43
MAE: 70.85
RMSE: 85.66
R²: -0.58


## RNN con funciones

In [11]:
X, y = create_sequences_rnn(df, target_column="valor_(GWh)", lookback=30)
X_train, X_val, y_train, y_val = train_test(X, y)

In [22]:
rnn, history_rnn = get_model_rnn(X, X_train, X_val, y_train, y_val, epochs=1000)

Epoch 1/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0681 - mae: 0.1909 - val_loss: 0.0091 - val_mae: 0.0729
Epoch 2/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0142 - mae: 0.0932 - val_loss: 0.0089 - val_mae: 0.0717
Epoch 3/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0125 - mae: 0.0853 - val_loss: 0.0086 - val_mae: 0.0698
Epoch 4/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0116 - mae: 0.0820 - val_loss: 0.0090 - val_mae: 0.0709
Epoch 5/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0104 - mae: 0.0776 - val_loss: 0.0101 - val_mae: 0.0755
Epoch 6/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0106 - mae: 0.0766 - val_loss: 0.0086 - val_mae: 0.0704
Epoch 7/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0090 - mae: 0.0709 - val_loss: 0.0101 - val_mae: 0.0758
Epoch 8/1000
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0092 - mae: 0.0723 - val_loss: 0.0090 - val_mae: 0.0724
Epoch 9/1000
129/129 ━━━━━━━━━━━━━━━━━━━

In [6]:
rnn = joblib.load("MODELS/RNN_LSTM/rnn.pkl")

In [8]:
# with open('MODELS/RNN_LSTM/history_rnn.pkl', 'wb') as f:
#     pkl.dump(history_rnn, f)

with open('MODELS/RNN_LSTM/history_rnn.pkl', 'rb') as f:
    history_rnn = pkl.load(f)

In [9]:
fig_rnn_mae = plot_mae(history_rnn)
fig_rnn_mae.show()

In [12]:
pred_real_rnn_1s, target_real_rnn_1s = predict_1step_rnn(X, y_val, rnn, scaler)
pred_real_rnn_multi, target_real_rnn_multi = predict_multi_rnn(X, X_val, y_val, rnn, scaler)
# pred_real_rnn_noise, target_real_rnn_noise = predict_noise(X_val, y_val, rnn, scaler)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━

In [26]:
fig_rnn_1step = plot_validation(pred_real_rnn_1s, target_real_rnn_1s)
fig_rnn_1step.show()

In [27]:
fig_rnn_multi = plot_validation(pred_real_rnn_multi, target_real_rnn_multi)
fig_rnn_multi.show()

In [26]:
# fig_rnn_noise = plot_validation(pred_real_rnn_noise, target_real_rnn_noise)
# fig_rnn_noise.show()

In [15]:
metricas_rnn_1step = pd.DataFrame.from_dict(metricas_rnn(target_real_rnn_1s, pred_real_rnn_1s))
metricas_rnn_multi = pd.DataFrame.from_dict(metricas_rnn(target_real_rnn_multi, pred_real_rnn_multi))
# metricas_rnn_noise = metricas(target_real_rnn_noise, pred_real_rnn_noise)

In [23]:
metricas_rnn = pd.concat([metricas_rnn_1step, metricas_rnn_multi], axis=0).reset_index(drop=True)

In [25]:
metricas_rnn['modelo'] = ['RNN', 'RNN']
metricas_rnn['prediccion'] = ['1-step', 'multi-step']

In [27]:
metricas_rnn.to_csv("MODELS/RNN_LSTM/metricas_rnn.csv")

In [20]:
pred_real_rnn_1step = prediction_1step_rnn(X, y_val, scaler, rnn, n_future=14)
pred_real_rnn_multi = prediction_multistep_rnn(X, y_val, scaler, rnn, n_future=14)
# pred_real_rnn_noise = prediction_noise(X, y_val, scaler, rnn, n_future=14)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [21]:
plot_n_future(pred_real_rnn_1step)

In [22]:
plot_n_future(pred_real_rnn_multi)

_______________________________

# LSTM

In [28]:
X, y = create_sequences_rnn(df, target_column="valor_(GWh)")
X_train, X_val, y_train, y_val = train_test(X, y)

In [29]:
lstm, history_lstm = get_model_lstm(X, X_train, X_val, y_train, y_val, epochs=1000, lookback=30)

Epoch 1/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0774 - mae: 0.2090 - val_loss: 0.0242 - val_mae: 0.1188
Epoch 2/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0231 - mae: 0.1230 - val_loss: 0.0189 - val_mae: 0.1052
Epoch 3/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0178 - mae: 0.1069 - val_loss: 0.0121 - val_mae: 0.0870
Epoch 4/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0114 - mae: 0.0828 - val_loss: 0.0087 - val_mae: 0.0726
Epoch 5/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0105 - mae: 0.0783 - val_loss: 0.0175 - val_mae: 0.1115
Epoch 6/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0092 - mae: 0.0725 - val_loss: 0.0075 - val_mae: 0.0664
Epoch 7/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0087 - mae: 0.0699 - val_loss: 0.0114 - val_mae: 0.0842
Epoch 8/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0081 - mae: 0.0680 - val_loss: 0.0078 - val_mae: 0.0697
Epoch 9/1000
128/128 ━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
lstm = joblib.load("MODELS/RNN_LSTM/lstm.pkl")

In [ ]:
with open('MODELS/RNN_LSTM/history_lstm.pkl', 'wb') as f:
    pkl.dump(history_lstm, f)

# with open('MODELS/RNN_LSTM/history_lstm.pkl', 'rb') as f:
#     history_lstm = pkl.load(f)

In [ ]:
fig_lstm_mae = plot_mae(history_lstm)

In [11]:
fig_lstm_mae.show()

In [ ]:
pred_real_lstm_1step, target_real_lstm_1step = predict_1step_rnn(X_val, y_val, lstm, scaler)
pred_real_lstm_multi, target_real_lstm_multi = predict_multi_rnn(X, X_val, y_val, rnn, scaler)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [ ]:
fig_lstm_1step = plot_validation(pred_real_lstm_1step, target_real_lstm_1step)
fig_lstm_1step.show()

In [ ]:
fig_lstm_multi = plot_validation(pred_real_lstm_multi, target_real_lstm_multi)
fig_lstm_multi.show()

In [ ]:
metricas_lstm_1step = pd.DataFrame(metricas_rnn(target_real_lstm_1step, pred_real_rnn_1s))
metricas_lstm_multi = pd.DataFrame(metricas_rnn(target_real_lstm_multi, pred_real_rnn_multi))

In [ ]:
metricas_lstm = pd.concat([metricas_lstm_1step, metricas_lstm_multi], axis=1)

In [ ]:
# model_lstm = joblib.load("lstm.pkl")

In [ ]:
pred_real_lstm_1step = prediction_1step_rnn(X, y_val, scaler, lstm, n_future=14)
pred_real_lstm_multi = prediction_multistep_rnn(X, y_val, scaler, lstm, n_future=14)

NameError: name 'lstm' is not defined

In [8]:
plot_n_future(pred_real_lstm_1step)

In [9]:
plot_n_future(pred_real_lstm_multi)

In [ ]:
metricas = pd.concat([metricas_rnn, metricas_lstm], axis=1)
